In [1]:
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
import scipy.sparse as sp

In [2]:
import torch
import random
import numpy as np

torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

# data

## pretrain vec

In [3]:
with open('./res_data/pretrain_vec','r') as f:
    pretrain_vec = np.loadtxt(f,delimiter=' ')

In [4]:
#pretrain_vec[0]

In [5]:
graphs_list = dgl.load_graphs('./res_data/ad_graphs.dgl')[0]

In [6]:
f_ad_index = open('./res_data/ad_index','rb')
ad_index = pickle.load(f_ad_index)

f_index_ad = open('./res_data/index_ad','rb')
index_ad = pickle.load(f_index_ad)

f_ds_ad_label = open('./res_data/ds_ad_label','rb')
ds_ad_label = pickle.load(f_ds_ad_label)

In [7]:
for i in [int(i[1]) for i in sorted(index_ad.items(),key = lambda x:x[0])]:
    #print(i)
    pass

In [8]:
for index,g in enumerate(graphs_list):
    graphs_list[index].edata['w'] =  graphs_list[index].edata['w'].view(-1,1)
    #graphs_list[index] = dgl.add_self_loop(graphs_list[index])
    pass

In [9]:
len(graphs_list)

62

In [10]:
g0 = graphs_list[0]

# generate label graph

In [11]:
def getNegGraph(g):
    u, v = g.edges()
    adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())),shape=(g.number_of_nodes(),g.number_of_nodes()))
    #print(adj.shape)
    adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
    neg_u, neg_v = np.where(adj_neg != 0)
    rand_index = np.random.permutation(len(neg_u))
    neg_u = neg_u[rand_index][:g.num_edges()*40]
    neg_v = neg_v[rand_index][:g.num_edges()*40]
    gn = dgl.graph(([],[]),num_nodes=g.number_of_nodes())
    gn.add_edges(u,v,{'w':g.edata['w'].view(-1)})
    gn.add_edges(neg_u, neg_v,{'w':torch.zeros(neg_u.shape[0])})
    #gn.add_edges(u,v,{'w':g.edata['w'].view(-1)})
    return gn

In [12]:
pos_graphs_list = []
label_graphs_list = []
for g in graphs_list:
    pos_graphs_list.append(g)
    #graphs_list[index] = dgl.add_self_loop(graphs_list[index])
    label_graphs_list.append(getNegGraph(g))
    pass

In [13]:
#label_graphs_list

# model

In [14]:
from dgl.nn import GraphConv
from dgl.nn import SAGEConv
from dgl.nn import GATConv

In [15]:
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GCN ,self).__init__()
        #self.conv1 = GraphConv(in_feats, h_feats)
        #self.conv2 = GraphConv(h_feats, num_classes)
        self.conv1 = SAGEConv(in_feats,h_feats,aggregator_type='mean',feat_drop=0)
        self.conv2 = SAGEConv(h_feats,h_feats,aggregator_type='mean',feat_drop=0)
        self.do = nn.Dropout(p=0.2)
        
    def forward(self, g, in_feat):
        edge_weight=g.edata['w']
        if model.training:
            #edge_weight = self.do(edge_weight)
            pass
        h = self.conv1(g, in_feat,edge_weight=edge_weight)
        #h = self.conv1(g, in_feat)
        '''
        with g.local_scope():
            g.ndata['h'] = in_feat
            msg_fn = dgl.function.u_mul_e('h', 'w', 'm')
            #msg_fn = dgl.function.copy_src('h', 'm')
            reduce_fn = dgl.function.mean('m', 'neigh')
            g.update_all(msg_fn,reduce_fn)
            h = g.ndata['neigh']+g.ndata['h']
            #print('after in_feat',h[0])
            #h = self.conv1(g, in_feat)
            h = F.relu(h)
            return h
        '''  
        h = F.tanh(h)
        #h = F.relu(h)
        #h = self.conv2(g, h,edge_weight=g.edata['w'])
        #h = self.conv2(g, h)
        #print('h',h[0])
        #print('h_norm',torch.norm(h,dim=1,keepdim=True))
        ##是否进行 归一化！！！！
        #h = h/(torch.norm(h,dim=1,keepdim=True))
        #print('after norm',h[0])
        return h

In [16]:
class embedGCN(nn.Module):
    def __init__(self, in_feats, h_feats,num_classes):
        super(embedGCN ,self).__init__()
        self.node_embed = nn.Embedding(in_feats, 128)
        self.W0 = nn.Linear(128, h_feats)
        
        self.gcn = GCN(h_feats, h_feats)
        
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)
        self.reset_param()
        #self.node_embed.weight.data = torch.nn.Parameter(torch.tensor(pretrain_vec).float())
        
    def forward(self, g,feat=None):
        if feat == None:
            new_feat = self.node_embed(torch.arange(start=0,end=g.num_nodes(),dtype=torch.int64))
            new_feat = self.W0(new_feat)
            #new_feat = F.sigmoid(new_feat)
            #new_feat = new_feat/(torch.norm(new_feat,dim=0))
        else:
            new_feat = feat
              
        #print(new_feat.shape)
        #print(new_feat[0])
        h = self.gcn(g,new_feat)
        return h
    
    def apply_edges(self,edges):
        h = edges.src['h']
        l = edges.dst['h']#-edges.dst['h']
        score = (h*l).sum(dim=-1).view(-1)
        
        if not model.training:
            print('dot score',score)
            print('cos score',torch.nn.functional.cosine_similarity(h,l,dim=1))
        
        rand_mask = (torch.rand(score.shape[0])>0.5)*1
        return {'edge_score':score}
    
    def edgeout1(self,g,feat):
        with g.local_scope():
            g.ndata['h'] = feat
            g.apply_edges(self.apply_edges)
            return g.edata['score']
    
    def edgeout(self,g):
        with g.local_scope():
            feat = self.forward(g)
            g.ndata['h'] = feat
            g.apply_edges(self.apply_edges)
            return g.edata['score']
        
    def message_func(self, edges):
        return {'e': edges.data['edge_score']}    
    
    def exp_msg(self,edges):
        #print('before_add_fix',torch.exp(edges.data['norm_msg']))
        #print('after_add_fix',torch.exp(edges.data['norm_msg'])+0.0001)
        
        return {'exp_msg': torch.exp(edges.data['norm_msg'])}
    
    def sum_msg(self, nodes):
        h = torch.sum( nodes.mailbox['exp_msg'], dim=1)
        #print('before fix zero',h)
        #h += torch.ones(h.shape[0])*0.00001
        #print('after fix zero',h)
        return {'edge_sum': h}
       
    def GCNOut(self,g):
        pass
    
    def edgeout2(self,g,debug=False):
        with g.local_scope():
            feat = self.forward(g)
            #feat = self.node_embed(torch.arange(start=0,end=g.num_nodes(),dtype=torch.int64))
            return self.edgeout3(g,feat,debug)       
        
    def edgeout3(self,g,feat,debug=False):
        with g.local_scope():
            #print('node_feat',feat[:100])
            g.ndata['h'] = feat
            #print('feat',feat.shape)
            
            rg = dgl.reverse(g,copy_edata=True)

            ##compute edge_score before exp->get edge_score
            rg.apply_edges(self.apply_edges)
            ##compute max_edge_score of neighbour-> max_msg
            rg.update_all(self.message_func, dgl.function.max('e', 'max_msg'))
            
            if debug:
                print('rg src_msg',rg.edata['edge_score'])
                print('rg max_msg of node ',rg.ndata['max_msg'])
            ##compute
            rg.apply_edges(dgl.function.e_sub_v('edge_score', 'max_msg', 'norm_msg'))
            
            if debug:
                print('rg norm edge  score',rg.edata['norm_msg'])
             
            ## get exp score -> exp_msg
            rg.apply_edges(self.exp_msg)
            
            if debug:
                print('rg exp norm edge  score',rg.edata['exp_msg'])
            
            rg.update_all(dgl.function.copy_e('exp_msg','exp_msg'),reduce_func=self.sum_msg)
            
            rg.apply_edges(dgl.function.e_div_v('exp_msg', 'edge_sum', 'm'))
                       
            if debug:
                print('rg sum_meg',rg.ndata['edge_sum'])
                print('rg final  score',rg.edata['m'])
            
            '''
            g.apply_edges(self.apply_edges)
            g.update_all(self.message_func, self.reduce_func)
            if debug:
                print('g src_msg',g.edata['score'])
            g.apply_edges(dgl.function.e_div_v('score', 'edge_sum', 'm'))
            if debug:
                print('g sum_meg',g.ndata['edge_sum'])  
                print('g edge score',g.edata['m'])
            ''' 
            return rg.edata['m']
    
    def reset_param(self):
        torch.nn.init.xavier_uniform_(self.node_embed.weight, gain=1)

In [17]:
class seqEmbedGCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(seqEmbedGCN ,self).__init__()
        self.embedGCN = embedGCN(in_feats, h_feats, h_feats)
        self.rnn = nn.LSTM(h_feats,h_feats,num_layers=1,batch_first=False)
        self.outlayer = nn.Linear(h_feats,num_classes)
        self.sfx = nn.Softmax(dim=2)
        #self.embedGCN.node_embed.weight.data = torch.nn.Parameter(torch.eye(in_feats, h_feats))
        
    def forward(self, graphs,debug=False):
        ##graphs: 
        seq_input = torch.stack([self.embedGCN(g) for g in graphs],0)
        #seq_input = torch.randn(2, 368, 16): num_of_graph[seq] , num_of_node[batch],num_of_dim
        if debug:
            print('input',seq_input.shape)

        output, (hn, cn) = self.rnn(seq_input)
        if debug:
            print('o',output.shape)
            print('h',hn.shape)
            print('c',cn.shape)
        ##output:2, 368, 16 ; num_of_graph[seq] , num_of_node[batch],num_of_dim
        output = self.outlayer(output)
        if debug:
            print('last_out',output.shape)
        return self.sfx(output)
    
    def getGCNout(self,graphs,debug=False):
        return self.embedGCN(g)
    
    def getGCNedge(self,graph,node_feat,debug=False):
        return self.embedGCN.edgeout3(graph,node_feat,debug)
    
    def getGCNedge2(self,graph,debug=False):
        return self.embedGCN.edgeout2(graph,debug)    
    

    def getLSTMout(self,graphs):
        seq_input = torch.stack([self.embedGCN(g) for g in graphs],0)
        output, (hn, cn) = self.rnn(seq_input)
        return output[0]

In [18]:
g0n = getNegGraph(g0)

In [19]:
model = seqEmbedGCN(g0.num_nodes(),16,50)

In [20]:
res = model.getGCNedge(g0n,model.getGCNout(g0))

/home/bingfa.zjq/workingspace/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1949: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [21]:
res.shape

torch.Size([114582])

In [22]:
for index,i,j,m,k,n in zip(range(res.shape[0]),g0.edges(form='all')[0],g0.edges(form='all')[1],\
                           res,[0.0177, 0.0087, 0.0316, 0.0112, 0.0207, 0.0329, 0.0519, 0.1916, 0.1023,
        0.0177, 0.0064, 0.0106, 1.0000, 0.1873, 0.1023, 0.1873, 0.0843, 0.0267,
        0.0085, 0.0078, 0.0329, 0.0316, 0.0087, 0.0207, 0.0045, 0.1916, 0.0099,
        0.0213, 1.0000, 0.0843, 0.0258, 0.0147, 0.0519, 0.0167],g0.edata['w']):
    #print(index,i,j,m,k,n)
    pass

In [23]:
res = model([g0,g0n])

In [24]:
res[0,0,:]

tensor([0.0202, 0.0228, 0.0208, 0.0234, 0.0155, 0.0151, 0.0211, 0.0199, 0.0213,
        0.0167, 0.0183, 0.0198, 0.0204, 0.0157, 0.0202, 0.0199, 0.0172, 0.0171,
        0.0201, 0.0246, 0.0238, 0.0185, 0.0206, 0.0245, 0.0203, 0.0177, 0.0197,
        0.0231, 0.0213, 0.0168, 0.0176, 0.0152, 0.0191, 0.0185, 0.0179, 0.0222,
        0.0243, 0.0212, 0.0224, 0.0211, 0.0207, 0.0160, 0.0215, 0.0202, 0.0155,
        0.0199, 0.0197, 0.0242, 0.0248, 0.0214], grad_fn=<SliceBackward0>)

# train

In [25]:
def seq_train(graphs_list, model,debug=False):
    optimizer = torch.optim.AdamW(model.parameters(),weight_decay=0, lr=0.01)
    best_val_acc = 0
    best_test_acc = 0
    ##node loss
    mse_loss = torch.nn.MSELoss()
    kl_loss = nn.KLDivLoss(reduction="batchmean")
    l1_loss = torch.nn.L1Loss()
    bce_loss = torch.nn.BCELoss()
    
    epochs = 501#1001
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
    
    
    
    for e in range(epochs):
        model.train()
        beg_pos = 0
        seq_len = 4
        
        for step in range(seq_len,len(graphs_list)+seq_len,seq_len):
            model.train()
            
            input_node_graph =graphs_list[beg_pos:step]
            input_edge_graph = [getNegGraph(g) for g in input_node_graph]

            if len(input_node_graph) != seq_len:
                break
 
            #print(input_edge_graph)

            ##get pred_edge_score
            pred_edge_scores = torch.cat([model.getGCNedge(edge_g,model.getGCNout(node_g)) \
                                    for  edge_g,node_g in zip(input_edge_graph,\
                                                              input_node_graph)])
            
            #print('pred_edge_score_shape',pred_edge_scores.shape)
            
            
            #edges_mask = torch.cat([(i.edata['w']!=0)*1 for i in input_node_graph])
            #print('edge_mask_shape',edges_mask.shape)
            
            label_edge_weight = torch.cat([i.edata['w'] for i in input_edge_graph])
            #print('label_edge_weight_shape',label_edge_weight.shape)
        
            
            #return
            ##edge_loss 
            pred_edge_scores = pred_edge_scores  + 1e-7
            pred_edge_scores = torch.log(pred_edge_scores)
            #print('log edge_score',edge_score)  

            edge_loss = -1*(label_edge_weight*pred_edge_scores ).sum()/(label_edge_weight.sum())
            
            pred_node_score = model(input_node_graph)

            label_node_score = torch.stack([i.ndata['y'] for i in input_node_graph],dim=0)
            node_loss = kl_loss(torch.log(pred_node_score),label_node_score)
            #return 
            
            
            #edge_loss = -1*(edge_mask*edge_score).sum()/(edge_mask.sum())

            #edge_loss = kl_loss(edge_score,edge_weight)

            #print('edge_loss',edge_loss)
            #print('node_vec',model.embedGCN.node_embed.weight)

            alpha = 0.9
            loss=alpha*(torch.abs(edge_loss-1.9)+1.9)+ (1-alpha)*node_loss


            # Backward
            optimizer.zero_grad()
            with torch.autograd.set_detect_anomaly(True):
                loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=2, norm_type=2)
            optimizer.step()

            if e % 10 == 0:
                #print('node_vec',model.node_embed.weight[1:])
                #print('grad',model.node_embed.weight.grad[1:])
                pass

            if e%(20) ==0 :
                scheduler.step()


            if e % 10 == 0:
                #print('eval')
                print('epoch:%d,step:%d,loss:%f，edge_loss:%f,node_loss:%f'%\
                      (e,step,loss,edge_loss,node_loss))
                model.eval()
                #edge_score = model.edgeout3(g0n,model(g))
                #print('edge_score',edge_score)  
                #print('sum_edge_score',edge_score.sum())
                #scheduler.step()
                #print('lr',optimizer.state_dict()['param_groups'][0]['lr'])


In [26]:
g = graphs_list[0]
model = seqEmbedGCN(g.num_nodes(),128,50)

In [ ]:
seq_train(graphs_list, model)

/home/bingfa.zjq/workingspace/anaconda3/lib/python3.7/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


epoch:0,step:4,loss:21.416206，edge_loss:5.818824,node_loss:161.792664


In [ ]:
out.shape

In [ ]:
model.eval()
#res=model.getGCNedge2(g0n,True)
#print(res)

In [ ]:
res=model.getGCNedge(g0n,model.getGCNout(g),True)
print(res)

In [ ]:
 for index,i,j,m,n in zip(range(res.shape[0]),g0n.edges(form='all')[0],g0n.edges(form='all')[1],\
                         model.getGCNedge(g0n,model.getGCNout(g)),g0n.edata['w']):
    print(index,i.numpy(),j.numpy(),index_ad[int(i.numpy())],index_ad[int(j.numpy())],m,n)
    pass

In [ ]:
g.edata['w'].view(-1)

In [ ]:
0.0758

In [ ]:
#model.embedGCN.node_embed(torch.tensor(0))

In [ ]:
b = model.embedGCN.node_embed(torch.tensor(ad_index['110000']))

# eval

In [ ]:
node_vec = model.getGCNout(g0)
def getsim(a,b):
    print('cos',torch.nn.functional.cosine_similarity(node_vec[ad_index[a]],node_vec[ad_index[b]],dim=0))
    print('dot',(node_vec[ad_index[a]]*node_vec[ad_index[b]]).sum())

In [ ]:
getsim('130200','120000')

In [ ]:
getsim('130200','130600')

In [ ]:
getsim('130200','130800')

In [ ]:
def get_cosvalue(ad_a,ad_b):
    a = model.embedGCN.node_embed(torch.tensor(ad_index[ad_a]))
    b = model.embedGCN.node_embed(torch.tensor(ad_index[ad_b]))
    return torch.nn.functional.cosine_similarity(a,b,dim=0)

In [ ]:
model.getGCNout(g0).shape

In [ ]:
def get_cosvalue2(ad_a,ad_b):
    a = model.getGCNout(graphs_list[-1])[ad_index[ad_a]]
    b = model.getGCNout(graphs_list[-1])[ad_index[ad_b]]
    return torch.nn.functional.cosine_similarity(a,b,dim=0)

In [ ]:
get_cosvalue2('440300','810000')

In [ ]:
get_cosvalue('440100','810000')

# dump vector

In [ ]:
with open('./gnn_embed/vector.txt','w') as f:
    np.savetxt(f,X=model.getGCNout(g0).detach().numpy(),delimiter='\t')

In [ ]:
with open('./gnn_embed/embed_vector.txt','w') as f:
    np.savetxt(f,X=model.embedGCN.node_embed.weight.data.detach().numpy(),delimiter='\t')

In [ ]:
for i in index_ad.values():
    #print(i)
    pass

# save

In [ ]:
import datetime 

In [ ]:
dt = datetime.datetime.now()
timestamp = "%s-%s-%s-%s-%s"%(dt.year,dt.month,dt.day,dt.hour,dt.minute)
ckPath = './model_res/%s'%(timestamp)  
print('timestamp',ckPath)

In [ ]:
torch.save(model.state_dict(), ckPath)